In [3]:
import qulacs
from qulacs import QuantumState, QuantumCircuit
from qulacs.state import partial_trace
from qulacs.gate import DenseMatrix
import numpy as np
import numpy.linalg as npl
from scipy.linalg import expm
import matplotlib.pyplot as plt
%matplotlib inline

In [35]:
# [0, k): Charlie
# [k, 2k): Alice
# [2k, n+k): Black hole -> [k, n+k)
class YoungBlackHole():
    def __init__(self, n, k, dynamics, depth=-1):
        self.n = n
        self.k = k
        self.size = n + k
        self.dynamics = dynamics
        self.depth=depth
        self.reset()
    def reset(self):
        n, k = self.n, self.k
        dynamics = self.dynamics
        depth = self.depth
        self.state = QuantumState(n + k)
        self.circuit = QuantumCircuit(n + k)
        for i in range(k):
            self.circuit.add_H_gate(i)
            self.circuit.add_CNOT_gate(i, i + k)
        if dynamics == "lrc":
            self.add_LRC(k, n+k, depth)
        elif dynamics == "haar":
            self.circuit.add_random_unitary_gate(list(range(k, n+k)))
        elif dynamics == "heisenberg":
            self.add_Heisenberg(k, n+k, depth)
    def update(self):
        self.circuit.update_quantum_state(self.state)
    def add_LRC(self, l, r, depth):
        assert depth>=0 
        for d in range(depth):
            for i in range(l + d % 2, r - 1, 2):
                self.circuit.add_random_unitary_gate([i, i + 1])
    def add_Heisenberg(self, l, r, t):
        size = r-l
        X = np.array([[0, 1],[1, 0]])
        Y = np.array([[0, -1j],[1j, 0]])
        Z = np.array([[1, 0],[0, -1]])
        J = [X, Y, Z]
        H = np.zeros((1<<size, 1<<size), dtype='complex128')
        for i in range(l, r-1):
            Jx = np.random.random()
            Jy = np.random.random()
            Jz = np.random.random()
            M = Jx*np.kron(X, X)+Jy*np.kron(Y, Y)+Jz*np.kron(Z, Z)
            if i > k:
                M = np.kron(np.identity(1<<(i-l)), M)
            if i+2 < r:
                M = np.kron(M, np.identity(1<<(r-i-2)))
            H+=M
        U = DenseMatrix(list(range(l, r)), expm(-1j*t*H))
        self.circuit.add_gate(U)
    # l1 norm
    def L1(self, l):
        n, k = self.n, self.k
        mat_size = pow(2, n+k-l)
        trace = partial_trace(self.state, list(range(n + k - l, n + k)))
        return npl.norm(trace.get_matrix() - np.identity(mat_size)/mat_size, 'nuc')
    # mutual information
    def MI(self, l):
        n, k = self.n, self.k
        AB = partial_trace(self.state, list(range(k, n + k - l)))
        A = partial_trace(self.state, list(range(k, n + k)))
        B = partial_trace(self.state, list(range(0, n + k -l)))
        return self.S(A)+self.S(B)-self.S(AB)
    # entropy
    def S(self, rho):
        if rho is None:
            return 0
        mat = rho.get_matrix()
        eig_vals = npl.eigvalsh(mat)
        return -(eig_vals * np.log2(eig_vals+0.000001)).sum()

In [36]:
# simulator for young black hole
def simulate(model, l_max, iter_num):
    data_MI = np.zeros((l_max + 1, iter_num))
    data_L1 = np.zeros((l_max + 1, iter_num))
    for l in range(l_max + 1):
        mat_size = pow(2, n+k-l)
        for i in range(iter_num):
            if i%10 == 0: print(f"l={l}:{i}-th iteration")
            state = QuantumState(model.size)
            model.update()
            data_MI[l][i] = model.MI(l)
            data_L1[l][i] = model.L1(l)
            model.reset()
    return data_MI, data_L1
def plot_graph(data):
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.plot(np.average(data, axis=1))
    return fig
def save_data(data, file_name):
    np.savetxt("data/"+file_name, data, delimiter=',')

In [52]:
n, k, l_max = 9, 1, 9
model = YoungBlackHole(n, k, 'heisenberg', 1)
data_MI, data_L1 = simulate(model, l_max, 5)

l=0:i-th iteration
l=1:i-th iteration
l=2:i-th iteration
l=3:i-th iteration
l=4:i-th iteration
l=5:i-th iteration
l=6:i-th iteration
l=7:i-th iteration
l=8:i-th iteration
l=9:i-th iteration
